In [1]:
#modifcado: Víctor V.
#importing the dependencies
#genera texto a partir de Alicia en el país de las marvillas.
import numpy
#redes neuronales secuenciales
from keras.models import Sequential
#capa fuertemente conectada, densa
from keras.layers import Dense
#tecnica para recortar y hacer que aprenda de nuevo
#sirve para evitar sobre entrenamiento
from keras.layers import Dropout
#rede neuronal LSTM(Long Short-Term Memory)
from keras.layers import LSTM
#Esta clase es esencial para guardar el estado de tu modelo durante el entrenamiento.
from keras.callbacks import ModelCheckpoint
#from keras.utils import np_utils
#Codificación One-Hot: Es una técnica para representar variables categóricas como vectores numéricos.
from sklearn.preprocessing import OneHotEncoder
#one-Hot
from tensorflow.keras.utils import to_categorical
import sys

In [3]:
# sube y lo convierte a minúscua
filename = "wonderland.txt"
#leer el texto
raw_text = open(filename).read()
#pasar el texto a minúscula
raw_text = raw_text.lower()

In [4]:
#crea un diccionario con caracteres únicos del lenguaje
#con su representación en número, diccionaro
chars = sorted(list(set(raw_text)))


In [5]:
# here we can see all the unique charecters in the book
print(chars)

['\n', ' ', '!', '(', ')', ',', '-', '.', ':', ';', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '’', '“', '”']


In [6]:
#carcter y su índice
#colca en un diccionario la letra y una posición
char_to_int = dict((c, i) for i, c in enumerate(chars))
print(char_to_int)

{'\n': 0, ' ': 1, '!': 2, '(': 3, ')': 4, ',': 5, '-': 6, '.': 7, ':': 8, ';': 9, '?': 10, 'a': 11, 'b': 12, 'c': 13, 'd': 14, 'e': 15, 'f': 16, 'g': 17, 'h': 18, 'i': 19, 'j': 20, 'k': 21, 'l': 22, 'm': 23, 'n': 24, 'o': 25, 'p': 26, 'q': 27, 'r': 28, 's': 29, 't': 30, 'u': 31, 'v': 32, 'w': 33, 'x': 34, 'y': 35, 'z': 36, '—': 37, '’': 38, '“': 39, '”': 40}


In [7]:
#realiza conteos
n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)

Total Characters:  13637
Total Vocab:  41


In [8]:
# prepara el data set para el part de entrada salida
seq_length = 100
dataX = []
dataY = []
#ciclo que aumenta de 1 en 1
for i in range(0, n_chars - seq_length, 1):
    #una porción de código 100
    seq_in = raw_text[i:i + seq_length]
    #print('seq in : ', seq_in)
    #caracter a caracter
    seq_out = raw_text[i + seq_length]
    #print('seq out : ', seq_out)

    dataX.append([char_to_int[char] for char in seq_in])
    #print(dataX)
    #número
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)

Total Patterns:  13537


In [9]:
print(dataY)

[1, 14, 25, 8, 1, 25, 24, 13, 15, 1, 25, 28, 1, 30, 33, 19, 13, 15, 1, 29, 18, 15, 1, 18, 11, 14, 1, 26, 15, 15, 26, 15, 14, 1, 19, 24, 30, 25, 1, 30, 18, 15, 1, 12, 25, 25, 21, 1, 18, 15, 28, 1, 29, 19, 29, 30, 15, 28, 1, 33, 11, 29, 1, 28, 15, 11, 14, 19, 24, 17, 5, 1, 12, 31, 30, 1, 19, 30, 1, 18, 11, 14, 1, 24, 25, 1, 26, 19, 13, 30, 31, 28, 15, 29, 1, 25, 28, 1, 13, 25, 24, 32, 15, 28, 29, 11, 30, 19, 25, 24, 29, 1, 19, 24, 1, 19, 30, 5, 1, 39, 11, 24, 14, 1, 33, 18, 11, 30, 1, 19, 29, 1, 30, 18, 15, 1, 31, 29, 15, 1, 25, 16, 1, 11, 1, 12, 25, 25, 21, 5, 40, 1, 30, 18, 25, 31, 17, 18, 30, 1, 11, 22, 19, 13, 15, 1, 39, 33, 19, 30, 18, 25, 31, 30, 1, 26, 19, 13, 30, 31, 28, 15, 29, 1, 25, 28, 1, 13, 25, 24, 32, 15, 28, 29, 11, 30, 19, 25, 24, 29, 10, 40, 1, 29, 25, 1, 29, 18, 15, 1, 33, 11, 29, 1, 13, 25, 24, 29, 19, 14, 15, 28, 19, 24, 17, 1, 19, 24, 1, 18, 15, 28, 1, 25, 33, 24, 1, 23, 19, 24, 14, 1, 3, 11, 29, 1, 33, 15, 22, 22, 1, 11, 29, 1, 29, 18, 15, 1, 13, 25, 31, 22, 14, 5,

In [10]:
# cambia X a  [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# representación one-hot
y = to_categorical(dataY)
X,y

(array([[[0.26829268],
         [0.53658537],
         [0.46341463],
         ...,
         [0.02439024],
         [0.73170732],
         [0.6097561 ]],
 
        [[0.53658537],
         [0.46341463],
         [0.31707317],
         ...,
         [0.73170732],
         [0.6097561 ],
         [0.02439024]],
 
        [[0.46341463],
         [0.31707317],
         [0.36585366],
         ...,
         [0.6097561 ],
         [0.02439024],
         [0.34146341]],
 
        ...,
 
        [[0.02439024],
         [0.36585366],
         [0.82926829],
         ...,
         [0.41463415],
         [0.26829268],
         [0.56097561]],
 
        [[0.36585366],
         [0.82926829],
         [0.36585366],
         ...,
         [0.26829268],
         [0.56097561],
         [0.36585366]],
 
        [[0.82926829],
         [0.36585366],
         [0.31707317],
         ...,
         [0.56097561],
         [0.36585366],
         [0.17073171]]]),
 array([[0., 1., 0., ..., 0., 0., 0.],
        [0., 0.,

In [20]:

# define el LSTM model
#Red secuencia
model = Sequential()
#agrega una capa al modelo secuencial
#Define una capa LSTM con 256 unidades ocultas.
#input_shape=(X.shape[1], X.shape[2]): Especifica la forma de entrada esperada para la capa LSTM.
#X.shape[1]Longitud de las secuencias (número de pasos de tiempo).
#X.shape[2] Número de características en cada paso de tiempo.
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
#corte brusco,Dropout es una técnica de regularización utilizada para prevenir el sobreajuste en redes neuronales
model.add(Dropout(0.2))
#capa Densa con la salida igual al tamaño de y ,activacion softmax
model.add(Dense(y.shape[1], activation='softmax'))
#cómo va a configurar la entropía,optimizador Adam redes neuronales, medida
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])

In [ ]:
#descripción de lared
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 256)                 │         264,192 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 41)                  │          10,537 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 274,729 (1.05 MB)

 Trainable params: 274,729 (1.05 MB)

 Non-trainable params: 0 (0.00 B)

In [21]:
# define the punto de verificación
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.keras" # Add .keras extension
#save_best_only=True: Solo guarda el modelo con el mejor valor de la métrica
#mode='min': Indica que se busca minimizar el valor de la métrica monitoreada (la pérdida).
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]



In [22]:
# entrena el modelo
#callbacks=callbacks_list: Lista de objetos de devolución de llamada (callbacks) para monitorear y modificar el entrenamiento.
model.fit(X, y, epochs=3, batch_size=2000, callbacks=callbacks_list)
#epochs=10, cuántas veces le a pasar el conjunto de entrenamiento
#model.fit(X, y, epochs=10, batch_size=2000, callbacks=callbacks_list)

Epoch 1/3
7/7 [==============================] - ETA: 0s - loss: 3.5320 - accuracy: 0.1664
Epoch 1: loss improved from inf to 3.53197, saving model to weights-improvement-01-3.5320.keras
7/7 [==============================] - 9s 1s/step - loss: 3.5320 - accuracy: 0.1664
Epoch 2/3
7/7 [==============================] - ETA: 0s - loss: 3.0593 - accuracy: 0.1497
Epoch 2: loss improved from 3.53197 to 3.05931, saving model to weights-improvement-02-3.0593.keras
7/7 [==============================] - 8s 1s/step - loss: 3.0593 - accuracy: 0.1497
Epoch 3/3
7/7 [==============================] - ETA: 0s - loss: 2.9908 - accuracy: 0.1838
Epoch 3: loss improved from 3.05931 to 2.99078, saving model to weights-improvement-03-2.9908.keras
7/7 [==============================] - 8s 1s/step - loss: 2.9908 - accuracy: 0.1838


In [23]:
#guarda el modelo
model.save('model.h5')


In [18]:
#guarda la red con pesos
#from keras.models import load_model
#"Esta línea de código en Python carga un modelo previamente guardado en un archivo HDF5.
#model =""
#model = load_model('saved weights small network/weights-improvement-20-2.6334.hdf5')

OSError: No file or directory found at saved weights small network/weights-improvement-20-2.6334.hdf5

In [24]:
#de caractres con índices, crea un diccionario(índice,caracter)
int_to_char = dict((i, c) for i, c in enumerate(chars))
#vector que me permite  recuperar la letra por el índice, posición
int_to_char

{0: '\n',
 1: ' ',
 2: '!',
 3: '(',
 4: ')',
 5: ',',
 6: '-',
 7: '.',
 8: ':',
 9: ';',
 10: '?',
 11: 'a',
 12: 'b',
 13: 'c',
 14: 'd',
 15: 'e',
 16: 'f',
 17: 'g',
 18: 'h',
 19: 'i',
 20: 'j',
 21: 'k',
 22: 'l',
 23: 'm',
 24: 'n',
 25: 'o',
 26: 'p',
 27: 'q',
 28: 'r',
 29: 's',
 30: 't',
 31: 'u',
 32: 'v',
 33: 'w',
 34: 'x',
 35: 'y',
 36: 'z',
 37: '—',
 38: '’',
 39: '“',
 40: '”'}

In [25]:
dataX

[[11,
  22,
  19,
  13,
  15,
  1,
  33,
  11,
  29,
  1,
  12,
  15,
  17,
  19,
  24,
  24,
  19,
  24,
  17,
  1,
  30,
  25,
  1,
  17,
  15,
  30,
  1,
  32,
  15,
  28,
  35,
  1,
  30,
  19,
  28,
  15,
  14,
  1,
  25,
  16,
  1,
  29,
  19,
  30,
  30,
  19,
  24,
  17,
  1,
  12,
  35,
  1,
  18,
  15,
  28,
  1,
  29,
  19,
  29,
  30,
  15,
  28,
  1,
  25,
  24,
  1,
  30,
  18,
  15,
  1,
  12,
  11,
  24,
  21,
  5,
  1,
  11,
  24,
  14,
  1,
  25,
  16,
  1,
  18,
  11,
  32,
  19,
  24,
  17,
  1,
  24,
  25,
  30,
  18,
  19,
  24,
  17,
  1,
  30,
  25],
 [22,
  19,
  13,
  15,
  1,
  33,
  11,
  29,
  1,
  12,
  15,
  17,
  19,
  24,
  24,
  19,
  24,
  17,
  1,
  30,
  25,
  1,
  17,
  15,
  30,
  1,
  32,
  15,
  28,
  35,
  1,
  30,
  19,
  28,
  15,
  14,
  1,
  25,
  16,
  1,
  29,
  19,
  30,
  30,
  19,
  24,
  17,
  1,
  12,
  35,
  1,
  18,
  15,
  28,
  1,
  29,
  19,
  29,
  30,
  15,
  28,
  1,
  25,
  24,
  1,
  30,
  18,
  15,
  1,
  12,
  11,
  24,
 

In [27]:
int_to_char[11]

'a'

In [32]:
#generación de texto
#aquí la red neuronal aprendió del texto de Alicia

# selecciona un índice del tamaño del datos
start = numpy.random.randint(0, len(dataX)-1)
#print(dataX)
start = 0
#inicia con el texto en indice calculado
pattern = dataX[start]
print ("Semilla:")
#obtiene el caracter de la posición
print ("\"", ' '.join([int_to_char[value] for value in pattern]), "\"")

# generar caracteres
for i in range(1000):
	# redimensiona un array pattern a una nueva forma tridimensional
	#1: El primer elemento indica que se crea un nuevo eje con una sola dimensión.
	#El segundo elemento especifica que la segunda dimensión tendrá la misma longitud que el array original pattern.
	#1: El tercer elemento indica que se crea un nuevo eje con una sola dimensión.
	x = numpy.reshape(pattern, (1, len(pattern), 1))
  #divide cada elemento del array x por el valor de n_vocab convertido a flotante.
	x = x / float(n_vocab)
 #realiza la predicción,no mostrando nada,verbose=1
	prediction = model.predict(x, verbose=0)
  #obtiene el índice del valor más alto en la predicciónms
	index = numpy.argmax(prediction)
  #el resultado lo coloca como índice y saca el caracter
	result = int_to_char[index]
  #guarda el caracter en una lista, de número a lista
	seq_in = [int_to_char[value] for value in pattern]
  #escribe el resultado
	#sys.stdout.write(result)

  #adiciona el índice a patter
	pattern.append(index)
 #pattern a una nueva lista que contiene todos los elementos de la lista original pattern excepto el primer elemento.
	pattern = pattern[1:len(pattern)]
print(result)
print ("\nDone.")

Semilla:
" a l i c e   w a s   b e g i n n i n g   t o   g e t   v e r y   t i r e d   o f   s i t t i n g   b y   h e r   s i s t e r   o n   t h e   b a n k ,   a n d   o f   h a v i n g   n o t h i n g   t o     "


KeyboardInterrupt: 

here we can see that the model performs very badly. an educated guess for this is that the model is too simple and is also trained for smaller epochs

In [33]:
# training larger model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])

In [34]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 100, 256)          264192    
                                                                 
 dropout_2 (Dropout)         (None, 100, 256)          0         
                                                                 
 lstm_3 (LSTM)               (None, 256)               525312    
                                                                 
 dropout_3 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 41)                10537     
                                                                 
Total params: 800041 (3.05 MB)
Trainable params: 800041 (3.05 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [35]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(X, y, epochs=10, batch_size=1300, callbacks=callbacks_list)

Epoch 1/10
11/11 [==============================] - ETA: 0s - loss: 3.2576 - accuracy: 0.1621
Epoch 1: loss improved from inf to 3.25756, saving model to weights-improvement-01-3.2576-bigger.hdf5
11/11 [==============================] - 24s 2s/step - loss: 3.2576 - accuracy: 0.1621
Epoch 2/10


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


11/11 [==============================] - ETA: 0s - loss: 2.9772 - accuracy: 0.1828
Epoch 2: loss improved from 3.25756 to 2.97716, saving model to weights-improvement-02-2.9772-bigger.hdf5
11/11 [==============================] - 21s 2s/step - loss: 2.9772 - accuracy: 0.1828
Epoch 3/10
11/11 [==============================] - ETA: 0s - loss: 2.9661 - accuracy: 0.1854
Epoch 3: loss improved from 2.97716 to 2.96605, saving model to weights-improvement-03-2.9661-bigger.hdf5
11/11 [==============================] - 21s 2s/step - loss: 2.9661 - accuracy: 0.1854
Epoch 4/10
11/11 [==============================] - ETA: 0s - loss: 2.9617 - accuracy: 0.1867
Epoch 4: loss improved from 2.96605 to 2.96171, saving model to weights-improvement-04-2.9617-bigger.hdf5
11/11 [==============================] - 21s 2s/step - loss: 2.9617 - accuracy: 0.1867
Epoch 5/10
11/11 [==============================] - ETA: 0s - loss: 2.9556 - accuracy: 0.1870
Epoch 5: loss improved from 2.96171 to 2.95562, saving m

In [ ]:
#training the model for another round of 10 epochs
model.fit(X, y, epochs=10, batch_size=1300, callbacks=callbacks_list)

Epoch 1/10
144415/144415 [==============================] - 3275s 23ms/step - loss: 2.3322 - acc: 0.3312
Epoch 2/10
144415/144415 [==============================] - 3333s 23ms/step - loss: 2.2832 - acc: 0.3450
Epoch 3/10
144415/144415 [==============================] - 3205s 22ms/step - loss: 2.2351 - acc: 0.3579
Epoch 4/10
144415/144415 [==============================] - 3310s 23ms/step - loss: 2.1945 - acc: 0.3696
Epoch 5/10
144415/144415 [==============================] - 3150s 22ms/step - loss: 2.1582 - acc: 0.3801
Epoch 6/10
144415/144415 [==============================] - 3139s 22ms/step - loss: 2.1219 - acc: 0.3895
Epoch 7/10
144415/144415 [==============================] - 3180s 22ms/step - loss: 2.0907 - acc: 0.3963
Epoch 8/10
144415/144415 [==============================] - 3342s 23ms/step - loss: 2.0630 - acc: 0.4049
Epoch 9/10
144415/144415 [==============================] - 3251s 23ms/step - loss: 2.0331 - acc: 0.4122
Epoch 10/10
144415/144415 [============================

In [36]:
#generating the text
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print ("\nDone.")

Seed:
" it; so, after hunting all about for it, he was obliged to write with one finger for the rest of the  "
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

We can see that generally there are fewer spelling mistakes and the text looks more realistic, but is still quite nonsensical.
try fitting a more complex network or try training this network for longer epochs in order to see considerable differences